# Variables

In [1]:
ga_input = '/grid/home/nbourgeois/data/test_proteins/pah/pah.pep.fas' #sequence pep.fas Input
tissue = 'Liver' # Tissue type for CoCoPuts
filename = 'bai_1'

# Imports


In [2]:
import os
cwd = os.getcwd()

## Import the code for codon opt
import sys
sys.path.insert(1, '/grid/home/nbourgeois/codonOpt')
# from run_pygad import *
from general_functions import *
from dyn_prog import *

## Metrics

In [3]:
cai_on = False
bai_on = True
cpg_on = False
pas_on = False # PolyA
sps_on = False # Splicing


# Data

In [4]:
(keys, seqs) = readFasta(ga_input)
if len(seqs) == 1:
    aa_seq=str(seqs[0])

# Genetic Algorithm

In [5]:
aa_seq_short = 'MST'

In [22]:
optimizer = Optimizer(aa_seq)


In [23]:
optimized_seq = optimizer.optimize()

In [24]:
get_bai(optimized_seq,optimizer.bai_weight_dict)

0.8920728745898466

In [25]:
optimized_seq

'ATGAGCACAGCCGTGCTGGAGAACCCTGGGCTGGGCAGGAAGCTGTCTGACTTTGGCCAGGAGACCTCCTACATCGAGGACAACTGCAACCAGAATGGAGCCATCTCCCTGATCTTCTCCCTGAAGGAGGAGGTGGGGGCCCTGGCCAAGGTGCTGCGCCTCTTCGAGGAGAATGATGTGAACCTGACCCACATCGAGAGCCGACCTTCCCGGCTGAAGAAGGATGAGTATGAATTCTTCACCCACCTGGACAAGAGGAGCCTGCCTGCCCTGACCAACATCATCAAGATCCTGCGCCACGACATCGGCGCCACCGTGCACGAGCTGAGCCGGGACAAGAAGAAGGACACCGTGCCCTGGTTTCCCAGGACCATCCAGGAGCTGGACCGCTTCGCCAACCAGATCCTGAGCTATGGTGCCGAGCTGGATGCTGACCACCCTGGCTTCAAGGACCCTGTGTACAGAGCCAGAAGGAAGCAGTTTGCTGACATTGCCTACAACTACCGCCATGGCCAGCCCATCCCCCGGGTGGAGTACATGGAGGAGGAGAAGAAGACCTGGGGCACCGTGTTCAAGACCCTGAAGAGCCTGTACAAGACCCACGCCTGCTATGAGTACAATCACATCTTCCCCCTGCTGGAGAAGTACTGTGGCTTCCATGAGGACAACATCCCCCAGCTGGAGGACGTGTCCCAGTTCCTGCAGACCTGCACTGGCTTCCGCCTGCGCCCTGTGGCTGGCCTGCTGAGCAGCCGGGACTTCCTGGGCGGCCTGGCCTTCCGAGTGTTCCACTGCACCCAGTACATCAGGCATGGCAGCAAGCCCATGTACACCCCAGAGCCAGACATCTGCCACGAGCTGCTGGGCCATGTGCCCCTGTTCTCTGACAGGAGCTTTGCCCAGTTCTCCCAGGAGATCGGCCTGGCCTCCCTGGGTGCCCCTGATGAGTACATCGAGAAGCTGGCCACCATCTACTGGTTCACCGTGGAGTTTGGCCTG